In [ ]:
#This script searches for new subs, copies raw T2w from processed iBEAT folder and raw qT1 from subject folder, and generates a binary mask. 
#Be sure to cheange filepaths 

In [4]:
from nipype.interfaces import freesurfer as fs
import pandas as pd
import os
from shutil import copyfile


/usr/local/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
crawfp = '/Volumes/iang/active/BABIES/BABIES_Crossectional/BABIES_Crossectional-T1'
lrawfp = '/Volumes/iang/active/BABIES/BABIES_Longitudinal/BABIES_Longitudinal-T3'
newfp= '/Volumes/iang/active/BABIES/BABIES_qT1/newproc'
ibeatdir = '/Volumes/iang/active/BABIES/BABIES_ibeat/processed_brains'

In [6]:
def getsubs_c():
    c_sub = pd.DataFrame(data = os.listdir(crawfp), columns = ["subs"])
    c_sub['subid'] = c_sub['subs'].map(lambda x: x.lstrip('._').rstrip('-C-T1'))
    c_sub['sub_final'] = c_sub['subid'].apply(lambda x: 'NaN' if '.txt' in x else x).apply(lambda x: 'NaN' if 'e' in x else x).apply(lambda x: 'NaN' if 'o' in x else x)
    c_subs = c_sub['sub_final'].tolist()
    return c_subs

def getsubs_l():
    l_sub = pd.DataFrame(data = os.listdir(lrawfp), columns = ["subs"])
    l_sub['subid'] = l_sub['subs'].map(lambda x: x.lstrip('._').rstrip('-L-T3'))
    l_sub['sub_final'] = l_sub['subid'].apply(lambda x: 'NaN' if '.txt' in x else x).apply(lambda x: 'NaN' if 'e' in x else x).apply(lambda x: 'NaN' if 'o' in x else x)
    l_subs = l_sub['sub_final'].tolist()
    return l_subs

In [ ]:
#mri_binarize --i 100-skullstripped_anat.nii --o binarized.nii --min 0 --max 1 --inv
csubs = getsubs_c()
lsubs = getsubs_l()

for sub in csubs:
    ibraw = ibeatdir+'/BABIES_C/processed_T2w'+'/'+sub+'-skullstripped.nii'
    subdir = newfp + '/'+ sub
    ibcopy = subdir+'/'+sub+'_skullstripped.nii'
    mask = subdir+'/'+sub+'-binarized_mask.nii'
    if os.path.exists(ibraw):
        print('iBEAT T2w exists for {}'.format(sub))
        if os.path.exists(subdir):
            print('Subject Directory exists for {}'.format(sub))
        else:
            print('Making subject directory for {}'.format(sub))
            os.makedirs(subdir)
    else:
        print('Processed T2w does not exist for {}'.format(sub))
    if os.path.exists(ibraw):
              if os.path.exists(ibcopy):
                  print('T2w already copied over for {}'.format(sub))
              else:
                  print('Copying over T2w for {}'.format(sub))
                  copyfile(ibraw, ibcopy)
    if os.path.exists(ibcopy):
              print('Computing Mask for {}'.format(sub))
              binvol = fs.Binarize(in_file = ibcopy, 
                      min=0,
                      max = 1,
                      invert = True,
                      binary_file= mask)
              binvol.run()
    else:
              print('Mask already computed for {}'.format(sub))
              
            
#     binvol = fs.Binarize(in_file = prac + '/100-skullstripped_anat.nii', 
#                       min=0,
#                       max = 1,
#                       invert = True,
#                       binary_file= prac + '/binarized_1.nii')
#     binvol.run()

In [ ]:
lsubs = getsubs_l()
for sub in lsubs:
    ibraw = ibeatdir+'/BABIES_LT3/processed_T2w'+'/'+sub+'-skullstripped.nii'
    subdir = newfp + '/'+ sub
    ibcopy = subdir+'/'+sub+'_skullstripped.nii'
    mask = subdir+'/'+sub+'-binarized_mask.nii'
    if os.path.exists(ibraw):
        print(('iBEAT T2w exists for {}'.format(sub))
        if os.path.exists(subdir):
            print('Subject Directory exists for {}'.format(sub))
        else:
            print('Making subject directory for {}'.format(sub))
            os.makedirs(subdir)
    else:
        print('Processed T2w does not exist for {}'.format(sub))
    if os.path.exists(ibraw):
              if os.path.exists(ibcopy):
                  print('T2w already copied over for {}'.format(sub))
              else:
                  print('Copying over T2w for {}'.format(sub))
                  copyfile(ibraw, ibcopy)
    if os.path.exists(ibcopy):
              print('Computing Mask for {}'.format(sub))
              binvol = fs.Binarize(in_file = ibcopy, 
                      min=0,
                      max = 1,
                      invert = True,
                      binary_file= mask)
              binvol.run()
    else:
              print('Mask already computed for {}'.format(sub))
              

In [17]:
csubs = getsubs_c()

def copyqt1_c():
    qt1raw_pe1 = crawfp +'/'+sub+'-C-T1/anat/qt1/recon/qt1_pe1.nii.gz'
    qt1raw_pe0 = crawfp+'/'+sub+'-C-T1/anat/qt1/recon/qt1_pe0.nii.gz'
    subdir = newfp + '/'+ sub
    if os.path.exists(subdir):
        if os.path.exists(qt1raw_pe0):
            if os.path.exists(qt1raw_pe1):
                if os.path.exists(subdir+'/'+sub+'_qt1_raw_pe0.nii.gz'):
                    print('Raw qT1 data already copied over{}, skipping'.format(sub))
                else:
                    print('Raw qT1 data exists for {}, copying over...'.format(sub))
                    copyfile(qt1raw_pe0, subdir+'/'+sub+'_qt1_raw_pe0.nii.gz')
                    copyfile(qt1raw_pe1, subdir+'/'+sub+'_qt1_raw_pe1.nii.gz')
                    print('Data successfully copied for {}'.format(sub))
            else:
                print('*****qT1 pe1 data not found for {}*****'.format(sub))
        else:
            print('*****qT1 pe0 data not found for {}*****'.format(sub))
    else:
        print('Subject Directory does not exist for {}, no T2w'.format(sub))

for sub in csubs:
    copyqt1_c()

*****qT1 pe0 data not found for 104*****
Subject Directory does not exist for NaN, no T2w
Subject Directory does not exist for 033x, no T2w
Subject Directory does not exist for 002, no T2w
Subject Directory does not exist for 09, no T2w
Raw qT1 data already copied over100, skipping
Subject Directory does not exist for 064, no T2w
*****qT1 pe0 data not found for 056*****
*****qT1 pe0 data not found for 067*****
Subject Directory does not exist for 020, no T2w
Raw qT1 data already copied over099, skipping
Subject Directory does not exist for 012, no T2w
Raw qT1 data already copied over064x, skipping
Subject Directory does not exist for 023, no T2w
Raw qT1 data already copied over110, skipping
Subject Directory does not exist for 028x, no T2w
Subject Directory does not exist for 045, no T2w
Raw qT1 data already copied over108, skipping
Subject Directory does not exist for 027, no T2w
Subject Directory does not exist for 122, no T2w
Subject Directory does not exist for 040, no T2w
Subject 

In [16]:
lsubs = getsubs_l()

def copyqt1_l():
    qt1raw_pe1 = lrawfp+'/'+sub+'-L-T3/anat/qt1/recon/qt1_pe1.nii.gz'
    qt1raw_pe0 = lrawfp+'/'+sub+'-L-T3/anat/qt1/recon/qt1_pe0.nii.gz'
    subdir = newfp + '/'+ sub
    if os.path.exists(subdir):
        if os.path.exists(qt1raw_pe0):
            if os.path.exists(qt1raw_pe1):
                if os.path.exists(subdir+'/'+sub+'_qt1_raw_pe0.nii.gz'):
                    print('Raw qT1 data already copied over{}, skipping'.format(sub))
                else:
                    print('Raw qT1 data exists for {}, copying over...'.format(sub))
                    copyfile(qt1raw_pe0, subdir+'/'+sub+'_qt1_raw_pe0.nii.gz')
                    copyfile(qt1raw_pe1, subdir+'/'+sub+'_qt1_raw_pe1.nii.gz')
                    print('Data successfully copied for {}'.format(sub))
            else:
                print('*****qT1 pe1 data not found for {}*****'.format(sub))
        else:
            print('*****qT1 pe0 data not found for {}*****'.format(sub))
    else:
        print('Subject Directory does not exist for {}, no T2w'.format(sub))
for sub in lsubs:
    copyqt1_l()

*****qT1 pe0 data not found for 1005*****
Raw qT1 data already copied over1036, skipping
Subject Directory does not exist for 1024, no T2w
Subject Directory does not exist for NaN, no T2w
Subject Directory does not exist for NaN, no T2w
Raw qT1 data already copied over1011, skipping
*****qT1 pe0 data not found for 1020*****
Raw qT1 data already copied over1027, skipping
*****qT1 pe0 data not found for 102*****
*****qT1 pe0 data not found for 1010*****
Subject Directory does not exist for NaN, no T2w
Subject Directory does not exist for NaN, no T2w
Subject Directory does not exist for 10, no T2w
Subject Directory does not exist for 1078, no T2w
Subject Directory does not exist for 1029, no T2w
Raw qT1 data already copied over1027, skipping
*****qT1 pe0 data not found for 102*****
*****qT1 pe0 data not found for 1010*****
Subject Directory does not exist for 10, no T2w
Subject Directory does not exist for 1078, no T2w
Subject Directory does not exist for 1029, no T2w
*****qT1 pe0 data no